In [1]:
import os
if os.system('nvidia-smi') == 0:
    import setGPU
import tensorflow as tf
import glob
import sys
import argparse
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
import resnet_v1_eembc
import yaml
import csv
# from keras_flops import get_flops # (different flop calculation)
import kerop
import training_pokemon as pk  #gets already formated pokemon dataset (images --> numpy arrays)
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop
random_crop_model = tf.keras.models.Sequential()
random_crop_model.add(RandomCrop(32, 32, input_shape=(32, 32, 3,)))

Wed Jul 27 15:02:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0 Off |                  N/A |
| 35%   28C    P8    N/A /  30W |     47MiB /  1998MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

2022-07-27 15:02:57.421827: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-27 15:02:57.441102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-07-27 15:02:57.441113: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-27 15:02:57.441451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN

In [2]:
# load dataset
X_train, y_train, X_test, y_test = pk.get_pokemon_dataset()
X_train, X_test = X_train/256., X_test/256.

y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
if loss == 'squared_hinge':
    y_train = y_train * 2 - 1  # -1 or 1 for hinge loss
    y_test = y_test * 2 - 1

Found 480 files belonging to 10 classes.
Found 480 files belonging to 10 classes.
Using 384 files for training.
Found 480 files belonging to 10 classes.
Using 96 files for validation.


2022-07-27 15:03:53.861727: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-07-27 15:03:53.918660: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-07-27 15:03:53.976941: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-07-27 15:03:54.089650: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-07-27 15:03:54.101614: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
2022-07-27 15:03:54.158727: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


NameError: name 'num_classes' is not defined

In [ ]:
# restore "best" model
model.load_weights(model_file_path)

# get predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

# evaluate with test dataset and share same prediction results
evaluation = model.evaluate(X_test, y_test)

auc = roc_auc_score(y_test, y_pred, average='weighted', multi_class='ovr')

print('Model test accuracy = %.3f' % evaluation[1])
print('Model test weighted average AUC = %.3f' % auc)